marketing_df : bank client data: 
https://www.kaggle.com/datasets/adityamhaske/bank-marketing-dataset?resource=download

1 - age (numeric)

2 - job : type of job (categorical:
"admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
"blue-collar","self-employed","retired","technician","services")

3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)

4 - education (categorical: "unknown","secondary","primary","tertiary")

5 - default: has credit in default? (binary: "yes","no")

6 - balance: average yearly balance, in euros (numeric)

7 - housing: has housing loan? (binary: "yes","no")

8 - loan: has personal loan? (binary: "yes","no")

* ##### related with the last contact of the current campaign:
9 - contact: contact communication type (categorical: "unknown","telephone","cellular")

10 - day: last contact day of the month (numeric)

11 - month: last contact month of year (categorical: "jan", "feb", "mar", …, "nov", "dec")

12 - duration: last contact duration, in seconds (numeric)

* ##### other attributes:
13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)

15 - previous: number of contacts performed before this campaign and for this client (numeric)

16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

* ##### Output variable (desired target):

17 - y - has the client subscribed a term deposit? (binary: "yes","no")

Missing Attribute Values: None



loan_df : Features:
https://www.kaggle.com/datasets/sahideseker/loan-default-prediction-dataset?utm_source=copilot.com&select=loan_default_prediction.csv

loan_id: Unique loan identifier

income: Monthly income of the applicant

loan_amount: Total amount of the loan

employment_status: Employment status (Employed / Unemployed)

default: Whether the loan was defaulted (1 = Yes, 0 = No)


INSEE API selected dataset 

1. Population 
   
    -'PTOT' : total legal population of the département (Private + collective housing),
   
    -'PCAP' : population of the préfecture city (eg : DEP‑75 → Paris, DEP‑95 → Cergy...),
   
    -'PMUN' : municipal population, meaning the **population living in private households** only is housing and urbanism indicators
selected dataset IDs

2. Income	DS_ERFS_MENAGE_SL	Revenu disponible et pauvreté
3. Unemployment 	DS_RP_EMPLOI_LR_PRINC	Population active et chômage
4. Housing	DS_RP_LOGEMENT_PRINC	Résidences principales
5. Age structure	DS_BTS_SAL_EQTP_SEX_AGE	Salaires par sexe et âge
6. Education	DS_RP_DIPLOMES_PRINC	Diplômes de la population

In [1]:
import numpy as np
import pandas as pd
import json

df_uci = pd.read_csv("../data/raw/bank_marketing.csv", sep=';')
df_loan = pd.read_csv("../data/raw/loan_default.csv")
df_uci

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


### 1. Explore and clean the UCI dataset

In [2]:
from functions import explore_dataset

explore_dataset(df_uci)


=== Dataset ===

Shape: (45211, 17)

Columns:
 ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

Info:
<class 'pandas.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age        45211 non-null  int64
 1   job        45211 non-null  str  
 2   marital    45211 non-null  str  
 3   education  45211 non-null  str  
 4   default    45211 non-null  str  
 5   balance    45211 non-null  int64
 6   housing    45211 non-null  str  
 7   loan       45211 non-null  str  
 8   contact    45211 non-null  str  
 9   day        45211 non-null  int64
 10  month      45211 non-null  str  
 11  duration   45211 non-null  int64
 12  campaign   45211 non-null  int64
 13  pdays      45211 non-null  int64
 14  previous   45211 non-null  int64
 15  poutcome   45211 non-null  s

Conclusion 
The UCI dataset looks clean and well-structured:

45,211 rows and 17 columns

No missing values

Columns are correctly typed (int and str)

In [3]:
from functions import clean_uci_dataset

df_uci_clean = clean_uci_dataset(df_uci)


Cleaning complete. Shape: (45211, 17)


In [4]:
df_uci_clean.info()
df_uci_clean.isna().sum()*100/len(df_uci_clean)


<class 'pandas.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age        45211 non-null  int64
 1   job        44923 non-null  str  
 2   marital    45211 non-null  str  
 3   education  43354 non-null  str  
 4   default    45211 non-null  str  
 5   balance    45211 non-null  int64
 6   housing    45211 non-null  str  
 7   loan       45211 non-null  str  
 8   contact    32191 non-null  str  
 9   day        45211 non-null  int64
 10  month      45211 non-null  str  
 11  duration   45211 non-null  int64
 12  campaign   45211 non-null  int64
 13  pdays      45211 non-null  int64
 14  previous   45211 non-null  int64
 15  poutcome   8252 non-null   str  
 16  y          45211 non-null  int64
dtypes: int64(8), str(9)
memory usage: 5.9 MB


age           0.000000
job           0.637013
marital       0.000000
education     4.107407
default       0.000000
balance       0.000000
housing       0.000000
loan          0.000000
contact      28.798301
day           0.000000
month         0.000000
duration      0.000000
campaign      0.000000
pdays         0.000000
previous      0.000000
poutcome     81.747805
y             0.000000
dtype: float64

In [5]:
df_uci_clean['contact_missing'] = df_uci_clean['contact'].isna().astype(int)
df_uci_clean['contact_missing'] 

0        1
1        1
2        1
3        1
4        1
        ..
45206    0
45207    0
45208    0
45209    0
45210    0
Name: contact_missing, Length: 45211, dtype: int64

In [6]:
from functions import handle_missing_values_uci

df_uci_final = handle_missing_values_uci(df_uci_clean)

df_uci_final.isna().sum()


Missing-value handling complete. Shape: (43193, 17)


age                0
job                0
marital            0
education          0
default            0
balance            0
housing            0
loan               0
contact            0
day                0
month              0
duration           0
campaign           0
pdays              0
previous           0
y                  0
contact_missing    0
dtype: int64

### 2. Explore and Clean the loan dataset

In [7]:
from functions import explore_dataset

explore_dataset(df_loan)

=== Dataset ===

Shape: (255347, 18)

Columns:
 ['LoanID', 'Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner', 'Default']

Info:
<class 'pandas.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   LoanID          255347 non-null  str    
 1   Age             255347 non-null  int64  
 2   Income          255347 non-null  int64  
 3   LoanAmount      255347 non-null  int64  
 4   CreditScore     255347 non-null  int64  
 5   MonthsEmployed  255347 non-null  int64  
 6   NumCreditLines  255347 non-null  int64  
 7   InterestRate    255347 non-null  float64
 8   LoanTerm        255347 non-null  int64  
 9   DTIRatio        255347 non-null  float64
 10  Education       255347 non-null  s

In [8]:
from functions import clean_loan_dataset

df_loan_clean = clean_loan_dataset(df_loan)
df_loan_clean.info()
df_loan_clean.head()


Loan dataset cleaned. Shape: (255347, 18)
<class 'pandas.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   loan_i_d          255347 non-null  str    
 1   age               255347 non-null  int64  
 2   income            255347 non-null  int64  
 3   loan_amount       255347 non-null  int64  
 4   credit_score      255347 non-null  int64  
 5   months_employed   255347 non-null  int64  
 6   num_credit_lines  255347 non-null  int64  
 7   interest_rate     255347 non-null  float64
 8   loan_term         255347 non-null  int64  
 9   d_t_i_ratio       255347 non-null  float64
 10  education         255347 non-null  str    
 11  employment_type   255347 non-null  str    
 12  marital_status    255347 non-null  str    
 13  has_mortgage      255347 non-null  str    
 14  has_dependents    255347 non-null  str    
 15  loan_purpose      255347 non-null  st

,loan_i_d,age,income,loan_amount,credit_score,months_employed,num_credit_lines,interest_rate,loan_term,d_t_i_ratio,education,employment_type,marital_status,has_mortgage,has_dependents,loan_purpose,has_co_signer,loan_default
0,i38pquqs96,56,85994,50587,520,80,4,15.23,36,0.44,bachelor's,full-time,divorced,yes,yes,other,yes,0
1,hpsk72wa7r,69,50432,124440,458,15,1,4.81,60,0.68,master's,full-time,married,no,no,other,yes,0
2,c1oz6dpj8y,46,84208,129188,451,26,3,21.17,24,0.31,master's,unemployed,divorced,yes,yes,auto,no,1
3,v2kksfm3un,32,31713,44799,743,0,3,7.07,24,0.23,high school,full-time,married,no,no,business,no,0
4,ey08jdhtzp,60,20437,9139,633,8,4,6.51,48,0.73,bachelor's,unemployed,divorced,no,yes,auto,no,0


### 3. Extract INSEE API DATA

In [9]:
from insee_api_functions import fetch_melodi_dataset
df_insee = fetch_melodi_dataset("DS_POPULATIONS_REFERENCE", "DEP-44")
df_insee

,GEO,FREQ,TIME_PERIOD,POPREF_MEASURE,OBS_VALUE_NIVEAU
0,2025-DEP-44,A,2023,PTOT,1517043.0
1,2025-DEP-44,A,2023,PCAP,29473.0
2,2025-DEP-44,A,2023,PMUN,1487570.0


In [10]:
from insee_api_functions import fetch_melodi_dataset

# Test extraction for Île-de-France (example: Paris)
df_test = fetch_melodi_dataset("DS_POPULATIONS_REFERENCE", "DEP-75")
df_test


,GEO,FREQ,TIME_PERIOD,POPREF_MEASURE,OBS_VALUE_NIVEAU
0,2025-DEP-75,A,2023,PMUN,2103778.0
1,2025-DEP-75,A,2023,PCAP,15634.0
2,2025-DEP-75,A,2023,PTOT,2119412.0


In [11]:
from insee_api_functions import fetch_dep_dataset

df_france = fetch_dep_dataset("DS_POPULATIONS_REFERENCE")
df_france.head()



,GEO,FREQ,TIME_PERIOD,POPREF_MEASURE,OBS_VALUE_NIVEAU,departement_code
0,2025-DEP-01,A,2023,PTOT,694945.0,DEP-01
1,2025-DEP-01,A,2023,PCAP,15601.0,DEP-01
2,2025-DEP-01,A,2023,PMUN,679344.0,DEP-01
3,2025-DEP-02,A,2023,PTOT,534380.0,DEP-02
4,2025-DEP-02,A,2023,PMUN,523342.0,DEP-02


In [12]:
df_france.info()

<class 'pandas.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GEO               84 non-null     str    
 1   FREQ              84 non-null     str    
 2   TIME_PERIOD       84 non-null     str    
 3   POPREF_MEASURE    84 non-null     str    
 4   OBS_VALUE_NIVEAU  84 non-null     float64
 5   departement_code  84 non-null     str    
dtypes: float64(1), str(5)
memory usage: 4.1 KB


In [13]:
# Mapping dictionary for readable département names

departement_name_map = {
    "DEP-01": "Ain", "DEP-02": "Aisne", "DEP-03": "Allier", "DEP-04": "Alpes-de-Haute-Provence",
    "DEP-05": "Hautes-Alpes", "DEP-06": "Alpes-Maritimes", "DEP-07": "Ardèche", "DEP-08": "Ardennes",
    "DEP-09": "Ariège", "DEP-10": "Aube", "DEP-11": "Aude", "DEP-12": "Aveyron", "DEP-13": "Bouches-du-Rhône",
    "DEP-14": "Calvados", "DEP-15": "Cantal", "DEP-16": "Charente", "DEP-17": "Charente-Maritime",
    "DEP-18": "Cher", "DEP-19": "Corrèze", "DEP-2A": "Corse-du-Sud", "DEP-2B": "Haute-Corse",
    "DEP-21": "Côte-d'Or", "DEP-22": "Côtes-d'Armor", "DEP-23": "Creuse", "DEP-24": "Dordogne",
    "DEP-25": "Doubs", "DEP-26": "Drôme", "DEP-27": "Eure", "DEP-28": "Eure-et-Loir", "DEP-29": "Finistère",
    "DEP-30": "Gard", "DEP-31": "Haute-Garonne", "DEP-32": "Gers", "DEP-33": "Gironde", "DEP-34": "Hérault",
    "DEP-35": "Ille-et-Vilaine", "DEP-36": "Indre", "DEP-37": "Indre-et-Loire", "DEP-38": "Isère",
    "DEP-39": "Jura", "DEP-40": "Landes", "DEP-41": "Loir-et-Cher", "DEP-42": "Loire", "DEP-43": "Haute-Loire",
    "DEP-44": "Loire-Atlantique", "DEP-45": "Loiret", "DEP-46": "Lot", "DEP-47": "Lot-et-Garonne",
    "DEP-48": "Lozère", "DEP-49": "Maine-et-Loire", "DEP-50": "Manche", "DEP-51": "Marne", "DEP-52": "Haute-Marne",
    "DEP-53": "Mayenne", "DEP-54": "Meurthe-et-Moselle", "DEP-55": "Meuse", "DEP-56": "Morbihan",
    "DEP-57": "Moselle", "DEP-58": "Nièvre", "DEP-59": "Nord", "DEP-60": "Oise", "DEP-61": "Orne",
    "DEP-62": "Pas-de-Calais", "DEP-63": "Puy-de-Dôme", "DEP-64": "Pyrénées-Atlantiques",
    "DEP-65": "Hautes-Pyrénées", "DEP-66": "Pyrénées-Orientales", "DEP-67": "Bas-Rhin", "DEP-68": "Haut-Rhin",
    "DEP-69": "Rhône", "DEP-70": "Haute-Saône", "DEP-71": "Saône-et-Loire", "DEP-72": "Sarthe",
    "DEP-73": "Savoie", "DEP-74": "Haute-Savoie", "DEP-75": "Paris", "DEP-76": "Seine-Maritime",
    "DEP-77": "Seine-et-Marne", "DEP-78": "Yvelines", "DEP-79": "Deux-Sèvres", "DEP-80": "Somme",
    "DEP-81": "Tarn", "DEP-82": "Tarn-et-Garonne", "DEP-83": "Var", "DEP-84": "Vaucluse", "DEP-85": "Vendée",
    "DEP-86": "Vienne", "DEP-87": "Haute-Vienne", "DEP-88": "Vosges", "DEP-89": "Yonne", "DEP-90": "Territoire de Belfort",
    "DEP-91": "Essonne", "DEP-92": "Hauts-de-Seine", "DEP-93": "Seine-Saint-Denis", "DEP-94": "Val-de-Marne",
    "DEP-95": "Val-d’Oise", "DEP-971": "Guadeloupe", "DEP-972": "Martinique", "DEP-973": "Guyane",
    "DEP-974": "La Réunion", "DEP-976": "Mayotte"
}

df_france["departement_name"] = df_france["departement_code"].map(departement_name_map)


In [14]:
# Rename all columns to match the format
df_france.rename(columns={
    "GEO": "geo_code",
    "FREQ": "frequency",
    "TIME_PERIOD": "year",
    "POPREF_MEASURE": "population_type",
    "OBS_VALUE_NIVEAU": "population_value",
    "departement_code": "departement_code",  # already correct
    "departement_name": "departement_name"   # newly added
}, inplace=True)


In [15]:
list(df_france['population_type'].unique())

['PTOT', 'PCAP', 'PMUN']

In [16]:
#df_france.to_csv("insee_population_by_departement.csv", index=False)


In [17]:
import requests, json

url = "https://api.insee.fr/melodi/catalog/all"
response = requests.get(url)
raw_data = json.loads(response.content)

print(type(raw_data))
print(len(raw_data))
print(raw_data[0])


<class 'dict'>
2


KeyError: 0

In [18]:
response.json()


{'message': 'Rate limit exceeded! You reached the limit of 30 requests per 1 minutes',
 'http_status_code': 429}

In [19]:
from insee_api_functions import list_melodi_datasets

catalog = list_melodi_datasets()
catalog.head()



,dataset_id,title_fr,description_fr


In [20]:
#Income datasets
catalog[catalog["title_fr"].str.contains("revenu", case=False, na=False)]


,dataset_id,title_fr,description_fr


In [21]:
# Unemployment datasets
catalog[catalog["title_fr"].str.contains("chômage", case=False, na=False)]

,dataset_id,title_fr,description_fr


In [22]:
# Unemployment datasets
catalog[catalog["title_fr"].str.contains("logement", case=False, na=False)]

,dataset_id,title_fr,description_fr


In [23]:
# Unemployment datasets
catalog[catalog["title_fr"].str.contains("âge", case=False, na=False)]

,dataset_id,title_fr,description_fr


In [24]:
# Unemployment datasets
catalog[catalog["title_fr"].str.contains("diplôme|scolarité|éducation", case=False, na=False)]

,dataset_id,title_fr,description_fr


In [25]:
from insee_api_functions import fetch_indicator_for_all_departements

df_income = fetch_indicator_for_all_departements("DS_ERFS_MENAGE_SL")
df_income.head()


Error for DEP-01: 429 Client Error: Too Many Requests for url: https://api.insee.fr/melodi/v1/data?dataset=DS_ERFS_MENAGE_SL&geo=DEP-01
Error for DEP-02: 429 Client Error: Too Many Requests for url: https://api.insee.fr/melodi/v1/data?dataset=DS_ERFS_MENAGE_SL&geo=DEP-02
Error for DEP-03: 429 Client Error: Too Many Requests for url: https://api.insee.fr/melodi/v1/data?dataset=DS_ERFS_MENAGE_SL&geo=DEP-03
Error for DEP-04: 429 Client Error: Too Many Requests for url: https://api.insee.fr/melodi/v1/data?dataset=DS_ERFS_MENAGE_SL&geo=DEP-04
Error for DEP-05: 429 Client Error: Too Many Requests for url: https://api.insee.fr/melodi/v1/data?dataset=DS_ERFS_MENAGE_SL&geo=DEP-05
Error for DEP-06: 429 Client Error: Too Many Requests for url: https://api.insee.fr/melodi/v1/data?dataset=DS_ERFS_MENAGE_SL&geo=DEP-06
Error for DEP-07: 429 Client Error: Too Many Requests for url: https://api.insee.fr/melodi/v1/data?dataset=DS_ERFS_MENAGE_SL&geo=DEP-07
Error for DEP-08: 429 Client Error: Too Many Req

KeyError: 'departement_code'

In [ ]:
catalog.columns